In [52]:
import pandas as pd
import yfinance as yf
import sqlalchemy
import datetime
import os
import time
import pandas_ta as ta
from backtesting import Backtest, Strategy
import numpy as np
from backtesting.lib import resample_apply
from backtesting.test import GOOG
from pandas_datareader import data

In [74]:
start_date = '2020-01-12'
end_date = '2023-01-11'
#df=yf.download('BTC-USD',start_date, end_date,interval='1d',auto_adjust=True )
df=yf.download('ASELS.IS',period="2y",interval='1d',auto_adjust=True)

[*********************100%***********************]  1 of 1 completed


In [75]:
df

,Open,High,Low,Close,Volume
Date,,,,,
2021-01-13,18.689442,18.768302,18.344438,18.423296,27993934
2021-01-14,18.521871,18.630300,18.176865,18.275438,24620292
2021-01-15,18.196577,18.285293,17.861430,17.940289,22196148
2021-01-18,17.920575,18.236008,17.674142,18.167007,24825552
2021-01-19,18.236007,18.472582,18.098005,18.147291,27918391
...,...,...,...,...,...
2023-01-09,64.500000,65.000000,58.000000,59.099998,108801844
2023-01-10,59.099998,60.799999,57.299999,58.150002,118780772
2023-01-11,58.200001,59.599998,52.599998,52.799999,101975581


In [95]:
import datetime
import pandas_ta as ta
import pandas as pd
import numpy as np
from backtesting import Backtest
from backtesting.lib import crossover, TrailingStrategy
from backtesting import Strategy
from backtesting.test import GOOG

def supertrend1(data):
    suptr=ta.supertrend(high= data.High.s,low= data.Low.s,close= data.Close.s, length=10, multiplier=1)
    return suptr.to_numpy().T[0:1]    
    

class Deneme(Strategy):

    def init(self):
        # Precompute the two moving averages
        self.suptr =self.I(supertrend1,self.data)
        #self.bbands=self.I(indicator,self.data)
    def next(self):
        sup=self.suptr[0]        
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if  self.data.Close[-2]<sup[-2] and self.data.Close[-1]>sup[-1]:
            #self.position.close()
            self.buy()
        # Else, if sma1 crosses below sma2, closeSany existing
        # long trades, and sell the asset
        elif self.data.Close[-2]>sup[-2] and self.data.Close[-1]<sup[-1]:
            self.position.close()
            #self.sell()
bt = Backtest(df, Deneme, cash=10_000, commission=.002)
stats = bt.run()
stats
bt.plot(plot_volume=False, plot_pl=False)

Row(id='3436', ...)

In [84]:
import datetime
import pandas_ta as ta
import pandas as pd

from backtesting import Backtest
from backtesting.lib import crossover, TrailingStrategy
from backtesting import Strategy
from backtesting.test import GOOG

def supertrend(data):
    suptr=ta.supertrend(high= self.data.High.s,low= self.data.Low.s, 
                            close= self.data.Close.s, length=10, multiplier=1.0)
    print(suptr)
    return suptr.to_numpy().T[:3]
#def sup2(fr):
#    return ta.supertrend(fr.High, fr.Low, fr.Close, length=10, multiplier=2.0)
#def sup3(fr):
#    return ta.supertrend(fr.High, fr.Low, fr.Close, length=10, multiplier=3.0)
def EMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

class deneme(Strategy):

    # Do as much initial computation as possible
    n1=5
    n2=10
    def init(self):
        
        #super().init()
        #super().set_trailing_sl(1)
        
        # Precompute the two moving averages
        self.ema5 = self.I(EMA, self.data.Close, self.n1)
        self.suptr =self.I(supertrend,self.data)
        print(self.suptr)
        pass
        #self.atr= self.I(ta.atr, high= self.data.High.s,low= self.data.Low.s, 
        #                    close= self.data.Close.s, length=10)
        #self.super1 = self.I(ta.supertrend, high= self.data.High.s,low= self.data.Low.s, 
        #                    close= self.data.Close.s, length=10, multiplier=1.0, plot=False, overlay=True)
        #self.super2 = self.I(ta.supertrend, high= self.data.High.s,low= self.data.Low.s, 
        #                     close= self.data.Close.s, length=10, multiplier=2.0,  plot=False, overlay=True)
        #self.super3 = self.I(ta.supertrend, high= self.data.High.s,low= self.data.Low.s, 
        #                     close= self.data.Close.s, length=10, multiplier=3.0,  plot=False, overlay=True)
        #self.super1_w = resample_apply(
        #    'W-FRI',sup1, self.data.df,plot=False)
        #self.super2_w = resample_apply(
        #    'W-FRI',sup2, self.data.df, plot=False)
        #self.super3_w = resample_apply(
        #    'W-FRI',sup3, self.data.df,plot=False)        
        self.ema5 = self.I(ta.ema, close= self.data.Close.s, length=5, plot=True)
        #self.ema20_w = resample_apply('W-FRI',ta.ema, self.data.Close.s, length=20, plot=False)
        self.ema50 = self.I(ta.ema, close= self.data.Close.s, length=50, plot=False)
        #self.ema50_w = resample_apply('W-FRI',ta.ema, self.data.Close.s, length=50, plot=False)
        #self.macd = self.I(ta.macd, close= self.data.Close.s, plot=False)
        #self.macd_w = resample_apply('W-FRI',ta.macd, self.data.Close.s, plot=False)
    # Step through bars one by one
    # Note that multiple buys are a thing here
    def next(self):
        pass
        ema5= self.ema5
        ema10=self.ema10
        #super().next()
        #price = self.data.Close.s
        #low= self.data.Low.s
        #atr = self.atr
        #super1=self.super1[1]
        #super2=self.super2[0]
        #super3=self.super3[0]
        #super1w=self.super1_w[0]
        #super2w=self.super2_w[0]
        #super3w=self.super3_w[0]
        
        #ema50= self.ema50
        #ema20w= self.ema20_w
        #ema50w= self.ema50_w
        #macdh=self.macd[1]
        #macdh_w=self.macd_w[1]

        if  crossover(self.data.Close, self.ema5):
            #and price[-1]>ema50[-1]:
                self.position.close()
                self.buy()
        elif crossover(self.ema5, self.data.Close):
                self.position.close()
                #self.sell()

bt = Backtest(GOOG, deneme, cash=1000000, commission=.002)

stats = bt.run()
#stats
bt.plot(plot_volume=False, plot_pl=True, plot_equity=False)

RuntimeError: Indicator "supertrend" errored with exception: name 'self' is not defined

In [54]:
from backtesting import Strategy
from backtesting.lib import crossover

def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()


class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 5
    n2 = 5
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.data.Close, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, closeSany existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.data.Close):
            self.position.close()
            #self.sell()
bt = Backtest(GOOG, SmaCross, cash=10_000, commission=.002)
stats = bt.run()
stats
bt.plot(plot_volume=False, plot_pl=False)

Row(id='23769', ...)

In [ ]:
help(ta.macd)

In [28]:
deneme=ta.macd(df.Close)
deneme

,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
2020-01-13,NaN,NaN,NaN
2020-01-14,NaN,NaN,NaN
2020-01-15,NaN,NaN,NaN
2020-01-16,NaN,NaN,NaN
2020-01-17,NaN,NaN,NaN
...,...,...,...
2023-01-05,-4.906985,-0.712439,-4.194546
2023-01-06,-4.575295,-0.304599,-4.270696
2023-01-09,-4.221005,0.039753,-4.260758
2023-01-10,-3.849056,0.329361,-4.178417


In [13]:
deneme2= ta.supertrend(df.High, df.Low, df.Close, length=10, multiplier=1.0)
deneme2

,SUPERT_10_1.0,SUPERTd_10_1.0,SUPERTl_10_1.0,SUPERTs_10_1.0
2019-01-03,0.000000,1,NaN,NaN
2019-01-04,NaN,1,NaN,NaN
2019-01-07,NaN,1,NaN,NaN
2019-01-08,NaN,1,NaN,NaN
2019-01-09,NaN,1,NaN,NaN
...,...,...,...,...
2022-10-12,228.163041,-1,NaN,228.163041
2022-10-13,227.764242,-1,NaN,227.764242
2022-10-14,227.764242,-1,NaN,227.764242
2022-10-17,227.764242,-1,NaN,227.764242


In [8]:
help(ta.ema)

Help on function ema in module pandas_ta.overlap.ema:

ema(close, length=None, talib=None, offset=None, **kwargs)
    Exponential Moving Average (EMA)
    
    The Exponential Moving Average is more responsive moving average compared to the
    Simple Moving Average (SMA).  The weights are determined by alpha which is
    proportional to it's length.  There are several different methods of calculating
    EMA.  One method uses just the standard definition of EMA and another uses the
    SMA to generate the initial value for the rest of the calculation.
    
    Sources:
        https://stockcharts.com/school/doku.php?id=chart_school:technical_indicators:moving_averages
        https://www.investopedia.com/ask/answers/122314/what-exponential-moving-average-ema-formula-and-how-ema-calculated.asp
    
    Calculation:
        Default Inputs:
            length=10, adjust=False, sma=True
        if sma:
            sma_nth = close[0:length].sum() / length
            close[:length - 1] = n

In [12]:
ema=ta.ema(close=df.Close, length=20)
ema

2020-01-13           NaN
2020-01-14           NaN
2020-01-15           NaN
2020-01-16           NaN
2020-01-17           NaN
                 ...    
2022-01-05    173.813966
2022-01-06    173.546684
2022-01-07    173.320955
2022-01-10    173.118618
2022-01-11    173.209200
Name: EMA_20, Length: 505, dtype: float64

In [16]:
bbands = ta.bbands(close = df.Close, std = 1)
bbands

,BBL_5_1.0,BBM_5_1.0,BBU_5_1.0,BBB_5_1.0,BBP_5_1.0
2019-01-03,NaN,NaN,NaN,NaN,NaN
2019-01-04,NaN,NaN,NaN,NaN,NaN
2019-01-07,NaN,NaN,NaN,NaN,NaN
2019-01-08,NaN,NaN,NaN,NaN,NaN
2019-01-09,20.726126,21.691867,22.657607,8.904171,0.953952
...,...,...,...,...,...
2022-10-12,215.799657,223.580005,231.360353,6.959789,0.092563
2022-10-13,217.449548,220.298004,223.146461,2.586003,0.749608
2022-10-14,210.328390,216.682004,223.035618,5.864459,-0.420106
2022-10-17,210.181759,215.960004,221.738248,5.351217,0.793342


In [57]:
help(ta.atr)

Help on function atr in module pandas_ta.volatility.atr:

atr(high, low, close, length=None, mamode=None, talib=None, drift=None, offset=None, **kwargs)
    Average True Range (ATR)
    
    Averge True Range is used to measure volatility, especially volatility caused by
    gaps or limit moves.
    
    Sources:
        https://www.tradingview.com/wiki/Average_True_Range_(ATR)
    
    Calculation:
        Default Inputs:
            length=14, drift=1, percent=False
        EMA = Exponential Moving Average
        SMA = Simple Moving Average
        WMA = Weighted Moving Average
        RMA = WildeR's Moving Average
        TR = True Range
    
        tr = TR(high, low, close, drift)
        if 'ema':
            ATR = EMA(tr, length)
        elif 'sma':
            ATR = SMA(tr, length)
        elif 'wma':
            ATR = WMA(tr, length)
        else:
            ATR = RMA(tr, length)
    
        if percent:
            ATR *= 100 / close
    
    Args:
        high (pd.Series):

In [14]:
atr=ta.atr(df.Close, df.High, df.Low, length=10)
atr

2020-01-13         NaN
2020-01-14         NaN
2020-01-15         NaN
2020-01-16         NaN
2020-01-17         NaN
                ...   
2022-01-05    4.601081
2022-01-06    4.469069
2022-01-07    4.270720
2022-01-10    3.989800
2022-01-11    4.287774
Name: ATRr_10, Length: 505, dtype: float64

In [49]:
from backtesting import Strategy
from backtesting.lib import crossover

def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()


class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            #self.sell()
bt = Backtest(df, SmaCross, cash=10_000, commission=.002)
stats = bt.run()
stats

Start                     2020-01-13 00:00:00
End                       2023-01-10 00:00:00
Duration                   1093 days 00:00:00
Exposure Time [%]                   58.278146
Equity Final [$]                 17480.717027
Equity Peak [$]                  24317.390466
Return [%]                           74.80717
Buy & Hold Return [%]                68.40217
Return (Ann.) [%]                   20.492543
Volatility (Ann.) [%]               28.891921
Sharpe Ratio                         0.709283
Sortino Ratio                        1.316286
Calmar Ratio                         0.697745
Max. Drawdown [%]                  -29.369677
Avg. Drawdown [%]                   -4.403719
Max. Drawdown Duration      372 days 00:00:00
Avg. Drawdown Duration       41 days 00:00:00
# Trades                                   16
Win Rate [%]                             50.0
Best Trade [%]                      80.599829
Worst Trade [%]                    -13.504434
Avg. Trade [%]                    

In [50]:
bt.plot(plot_volume=False, plot_pl=False)

Row(id='23551', ...)

In [ ]:
        if (price[-1]>ema20w[-1] or price[-1]>ema50w[-1])\
        and (price[-1]>super1w[-1] or price[-1]>super2w[-1] or price[-1]>super3w[-1]):
            if  price[-2]<super1[-2] and price[-1]>super1[-1]:
                #self.position.close()
                #self.buy(size=1, sl=price[-1]-2*atr[-1])
                self.buy(size=1)
        if price[-1]<ema50[-1]:
                self.position.close()
                #self.sell()